In [ ]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv('sample.csv', index_col=0)

In [ ]:
from elasticsearch import Elasticsearch

# Elasticsearch 클라이언트 설정 및 연결
es = Elasticsearch(
    ['https://localhost:9200'],
    http_auth=('elastic', 'password'),
    verify_certs=True,  # False로 아래 인증서 경로 없이 할 수 있으나 권장 X
    ca_certs='config/certs/http.p12'
)

In [ ]:
# Elasticsearch에 데이터 적재
for index, row in df.iterrows():
    es.index(index='index_name', id=index, body=row.to_dict())

In [ ]:
# BULK API를 사용하면 대용량 파일도 더욱 빨리 적재할 수 있다.

from elasticsearch import helpers

# Bulk API를 사용한 데이터 적재
actions = [
    {
        "_index": "index_name",
        "_id": index,
        "_source": row.to_dict()
    }
    for index, row in df.iterrows()
]

# Bulk API 실행
success, _ = helpers.bulk(es, actions)

In [ ]:
# df의 'Latitude'열과 'Longitude'열로 'geo_point'의 'location'열 추가하는 법 (kibana 대시보드의 GPS 표시를 위해)

# 우선 기존 인덱스에서 데이터 가져온다.
response = es.search(index='bike', body={"query": {"match_all": {}}})

# 'Latitude'열과 'Longitude'열을 활용한 geo_point의 'location'열 추가 생성하고 덮어쓰기
actions = []
for hit in response['hits']['hits']:
    source = hit['_source']
    location = {"lat": source['Latitude'], "lon": source['Longitude']}
    source['location'] = location
    action = {
        "_index": "gps_bike",
        "_id": hit['_id'],
        "_source": source
    }
    actions.append(action)

# Bulk API를 사용하여 기존 인덱스에 덮어쓰기
helpers.bulk(es, actions)
